In [ ]:
%load_ext autoreload
%autoreload 1

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K
from tqdm import tqdm

import sys
sys.path += ['../']
from utils.lr_finder import LRFinder

# Fake Model for Quick POC

In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K
from tqdm import tqdm

import sys
sys.path += ['../']
from utils.lr_finder import LRFinder

np.random.seed(43)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(1),
])

epoch = 5
batch_size = 32
sample_size = 50
train_data = tf.data.Dataset.from_tensor_slices((np.zeros((sample_size, batch_size, 10, 3)),np.random.rand(sample_size, batch_size, 1)))
test_data = tf.data.Dataset.from_tensor_slices((np.zeros((sample_size, batch_size, 10, 3)), np.random.rand(sample_size, batch_size, 1)))

callbacks = [LRFinder(train_data, batch_size, window_size=4, max_steps=400, filename='logs/lr_finder')]
lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay((4,400), epoch * len(train_data), alpha=1e-2)
opt = tf.keras.optimizers.Adam(learning_rate=lr_decayed_fn, beta_1=0.9, beta_2=0.98, epsilon=1e-09)
model.compile(optimizer=opt, loss='mse')

model.fit(train_data, validation_data=test_data, batch_size=batch_size, epochs=5, callbacks=callbacks)

2022-11-09 06:15:40.830794: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 06:15:40.838135: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


LRFinder: Finding `best_base_lr` range from: [5e-06(0.00000500), 1e-02(0.01000000)]...


LRFinder: logging loss:   0%|          | 0/400 [00:00<?, ?it/s]

ValueError: Weights for model sequential have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.

# Real Model for Validation and Test

In [2]:
%load_ext autoreload
%autoreload 1

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K
from tqdm import tqdm

import sys
sys.path += ['../']
from utils.lr_finder import LRFinder

import os, json, time
from datasets import get_dataset
from trainers import get_trainer

def save_log(history, val_metric: str=None):
    logs = { 'history': history }

    if val_metric != None:
        logs['best_acc'] = max(history[val_metric])
        print('Best score: ', logs['best_acc'])

    logfile_name = f'logs/qnet-{int(time.time())}.json'
    os.makedirs(os.path.dirname(logfile_name), exist_ok=True)
    with open(logfile_name, 'w') as f:
        json.dump(logs, f, indent=4)
    print('Log file saved at: ', logfile_name)

class Args:
    lr = 3e-4 * 128
    batch_size = 128
    seq_len = 8
    epochs = 5
    model = 'fnet'
    embed_size = 16
    num_blocks = 1
    qnet_depth = 1
    lr_finder = [4,400,'logs/lr_finder']
    dataset = 'msra'
    # dataset = 'colbert'
    # dataset = 'rentrunway'
    # dataset = 'msra'

args = Args()

np.random.seed(42)
tf.random.set_seed(42)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

dataset = get_dataset(args.dataset)
trainer = get_trainer(dataset.getTask())

fitting = trainer.train(args, dataset)

if dataset.getTask() == 'classification':
    if dataset.getOutputSize() > 2:
        save_log(fitting.history, 'val_categorical_accuracy')
    else:
        save_log(fitting.history, 'val_binary_accuracy')
else:
    save_log(fitting.history)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Version:  2.7.0
Eager mode:  True
GPU is NOT AVAILABLE
Num Replicas In Sync:  1


2022-11-11 18:06:52.020167: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Vocab size: 4749


2022-11-11 18:07:59.295955: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 f_net_1 (FNet)              (None, 8, 16)             76720     
                                                                 
 time_distributed_1 (TimeDis  (None, 8, 7)             119       
 tributed)                                                       
                                                                 
Total params: 76,839
Trainable params: 76,839
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1851/1851 [==============================] - 63s 32ms/step - loss: 0.3685 - accuracy: 0.8973 - val_loss: 0.3103 - val_accuracy: 0.9055 - val_f1_score: 0.0129
Epoch 2/5
1851/1851 [==============================] - 57s 29ms/step - loss: 0.2926 - accuracy: 0.9102 - val_loss: 0.2858 - val_accuracy: 0.9118 - val_f1_score: 0.0133
Epoch 3/5
1851/1851 [==================